<a href="https://colab.research.google.com/github/Monikahutapea/Student_Performance_Analysis/blob/main/Student_Performance_Analysis_using_Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Library

In [ ]:
# !pip install feature-engine
# !pip install matplotlib


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from feature_engine.outliers import Winsorizer
from sklearn.impute import SimpleImputer
from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder,LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neighbors import KNeighborsClassifier


In [ ]:
data = pd.read_csv("student.csv")
data

,Unnamed: 0,Id,Student_Age,Sex,High_School_Type,Scholarship,Additional_Work,Sports_activity,Transportation,Weekly_Study_Hours,Attendance,Reading,Notes,Listening_in_Class,Project_work,Grade
0,0,5001,21,Male,Other,50%,Yes,No,Private,0,Always,Yes,Yes,No,No,AA
1,1,5002,20,Male,Other,50%,Yes,No,Private,0,Always,Yes,No,Yes,Yes,AA
2,2,5003,21,Male,State,50%,No,No,Private,2,Never,No,No,No,Yes,AA
3,3,5004,18,Female,Private,50%,Yes,No,Bus,2,Always,No,Yes,No,No,AA
4,4,5005,22,Male,Private,50%,No,No,Bus,12,Always,Yes,No,Yes,Yes,AA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,140,5141,22,Female,State,50%,Yes,Yes,Private,0,Always,No,Yes,No,Yes,CC
141,141,5142,18,Female,State,75%,No,No,Private,0,Never,No,Yes,Yes,No,CC
142,142,5143,18,Female,Private,75%,No,No,Private,0,Always,Yes,No,No,No,AA
143,143,5144,22,Female,State,75%,Yes,Yes,Bus,12,Sometimes,No,Yes,No,Yes,CB


In [ ]:
data.dtypes

Unnamed: 0             int64
Id                     int64
Student_Age            int64
Sex                   object
High_School_Type      object
Scholarship           object
Additional_Work       object
Sports_activity       object
Transportation        object
Weekly_Study_Hours     int64
Attendance            object
Reading               object
Notes                 object
Listening_in_Class    object
Project_work          object
Grade                 object
dtype: object

In [ ]:
data.describe()

,Unnamed: 0,Id,Student_Age,Weekly_Study_Hours
count,145.000000,145.000000,145.000000,145.000000
mean,72.000000,5073.000000,19.682759,2.331034
std,42.001984,42.001984,1.992010,4.249273
min,0.000000,5001.000000,18.000000,0.000000
25%,36.000000,5037.000000,18.000000,0.000000
50%,72.000000,5073.000000,19.000000,0.000000
75%,108.000000,5109.000000,21.000000,2.000000
max,144.000000,5145.000000,26.000000,12.000000


In [ ]:
numerical_features = ['Student_Age', 'Weekly_Study_Hours']

for feature in numerical_features:
    plt.figure(figsize=(6, 4))
    sns.histplot(data=data, x=data[feature], bins=20, kde=True)
    plt.title(f'Distribution of {feature}')
    plt.xlabel(feature)
    plt.ylabel('Count')
    plt.show()

<ipython-input-62-2673a1d6d8b4>:4: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(figsize=(6, 4))


In [ ]:
categorical_features = ['Sex', 'High_School_Type', 'Scholarship', 'Additional_Work',
                        'Sports_activity', 'Transportation', 'Attendance', 'Reading',
                        'Notes', 'Listening_in_Class', 'Project_work', 'Grade']

for feature in categorical_features:
    plt.figure(figsize=(4, 3))
    sns.countplot(data=data, x=feature, order=data[feature].value_counts().index)
    plt.title(f'Count of {feature}')
    plt.xlabel(feature)
    plt.ylabel('Count')
    plt.xticks(rotation=45)
    plt.show()

In [ ]:
data.nunique()

Unnamed: 0            145
Id                    145
Student_Age             9
Sex                     2
High_School_Type        3
Scholarship             4
Additional_Work         2
Sports_activity         2
Transportation          2
Weekly_Study_Hours      4
Attendance              4
Reading                 2
Notes                   3
Listening_in_Class      3
Project_work            2
Grade                   8
dtype: int64

# Missing Value

## Cek Missing Value

In [ ]:
data.isnull().sum()

Unnamed: 0            0
Id                    0
Student_Age           0
Sex                   0
High_School_Type      0
Scholarship           1
Additional_Work       0
Sports_activity       0
Transportation        0
Weekly_Study_Hours    0
Attendance            0
Reading               0
Notes                 0
Listening_in_Class    0
Project_work          0
Grade                 0
dtype: int64

In [ ]:
cat_imputer = SimpleImputer(strategy="most_frequent")
category_kolom = ["Scholarship"]
data_kolom = data[category_kolom]
data_imputed = cat_imputer.fit_transform(data_kolom)
data_imputed_df = pd.DataFrame(data_imputed, columns=category_kolom)
data = pd.concat([data.drop(category_kolom, axis=1), data_imputed_df], axis=1)

In [ ]:
data.isnull().sum()

Unnamed: 0            0
Id                    0
Student_Age           0
Sex                   0
High_School_Type      0
Additional_Work       0
Sports_activity       0
Transportation        0
Weekly_Study_Hours    0
Attendance            0
Reading               0
Notes                 0
Listening_in_Class    0
Project_work          0
Grade                 0
Scholarship           0
dtype: int64

# Encoder

In [ ]:
kategori_urutan = [['Male', 'Female']]
encoder = OrdinalEncoder(categories=kategori_urutan)
data['Sex'] = encoder.fit_transform(data[['Sex']])

data

,Unnamed: 0,Id,Student_Age,Sex,High_School_Type,Additional_Work,Sports_activity,Transportation,Weekly_Study_Hours,Attendance,Reading,Notes,Listening_in_Class,Project_work,Grade,Scholarship
0,0,5001,21,0.0,Other,Yes,No,Private,0,Always,Yes,Yes,No,No,AA,50%
1,1,5002,20,0.0,Other,Yes,No,Private,0,Always,Yes,No,Yes,Yes,AA,50%
2,2,5003,21,0.0,State,No,No,Private,2,Never,No,No,No,Yes,AA,50%
3,3,5004,18,1.0,Private,Yes,No,Bus,2,Always,No,Yes,No,No,AA,50%
4,4,5005,22,0.0,Private,No,No,Bus,12,Always,Yes,No,Yes,Yes,AA,50%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,140,5141,22,1.0,State,Yes,Yes,Private,0,Always,No,Yes,No,Yes,CC,50%
141,141,5142,18,1.0,State,No,No,Private,0,Never,No,Yes,Yes,No,CC,75%
142,142,5143,18,1.0,Private,No,No,Private,0,Always,Yes,No,No,No,AA,75%
143,143,5144,22,1.0,State,Yes,Yes,Bus,12,Sometimes,No,Yes,No,Yes,CB,75%


In [ ]:
data.value_counts("High_School_Type")

High_School_Type
State      103
Private     25
Other       17
Name: count, dtype: int64

In [ ]:
kategori_urutan = [['State', 'Private', 'Other']]
encoder = OrdinalEncoder(categories=kategori_urutan)
data['High_School_Type'] = encoder.fit_transform(data[['High_School_Type']])

data

,Unnamed: 0,Id,Student_Age,Sex,High_School_Type,Additional_Work,Sports_activity,Transportation,Weekly_Study_Hours,Attendance,Reading,Notes,Listening_in_Class,Project_work,Grade,Scholarship
0,0,5001,21,0.0,2.0,Yes,No,Private,0,Always,Yes,Yes,No,No,AA,50%
1,1,5002,20,0.0,2.0,Yes,No,Private,0,Always,Yes,No,Yes,Yes,AA,50%
2,2,5003,21,0.0,0.0,No,No,Private,2,Never,No,No,No,Yes,AA,50%
3,3,5004,18,1.0,1.0,Yes,No,Bus,2,Always,No,Yes,No,No,AA,50%
4,4,5005,22,0.0,1.0,No,No,Bus,12,Always,Yes,No,Yes,Yes,AA,50%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,140,5141,22,1.0,0.0,Yes,Yes,Private,0,Always,No,Yes,No,Yes,CC,50%
141,141,5142,18,1.0,0.0,No,No,Private,0,Never,No,Yes,Yes,No,CC,75%
142,142,5143,18,1.0,1.0,No,No,Private,0,Always,Yes,No,No,No,AA,75%
143,143,5144,22,1.0,0.0,Yes,Yes,Bus,12,Sometimes,No,Yes,No,Yes,CB,75%


In [ ]:
data.value_counts("Additional_Work")

Additional_Work
No     96
Yes    49
Name: count, dtype: int64

In [ ]:
kolom_tertentu = ['Additional_Work', 'Sports_activity', 'Reading', 'Project_work', 'Notes', 'Listening_in_Class', 'Scholarship']

# Inisialisasi LabelEncoder
label_encoder = LabelEncoder()

# Loop melalui setiap kolom dan lakukan label encoding
for kolom in kolom_tertentu:
    data[kolom] = label_encoder.fit_transform(data[kolom])

data

,Unnamed: 0,Id,Student_Age,Sex,High_School_Type,Additional_Work,Sports_activity,Transportation,Weekly_Study_Hours,Attendance,Reading,Notes,Listening_in_Class,Project_work,Grade,Scholarship
0,0,5001,21,0.0,2.0,1,0,Private,0,Always,1,2,1,0,AA,2
1,1,5002,20,0.0,2.0,1,0,Private,0,Always,1,1,2,1,AA,2
2,2,5003,21,0.0,0.0,0,0,Private,2,Never,0,1,1,1,AA,2
3,3,5004,18,1.0,1.0,1,0,Bus,2,Always,0,2,1,0,AA,2
4,4,5005,22,0.0,1.0,0,0,Bus,12,Always,1,1,2,1,AA,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,140,5141,22,1.0,0.0,1,1,Private,0,Always,0,2,1,1,CC,2
141,141,5142,18,1.0,0.0,0,0,Private,0,Never,0,2,2,0,CC,3
142,142,5143,18,1.0,1.0,0,0,Private,0,Always,1,1,1,0,AA,3
143,143,5144,22,1.0,0.0,1,1,Bus,12,Sometimes,0,2,1,1,CB,3


In [ ]:
kategori_urutan = [['Private', 'Bus']]
encoder = OrdinalEncoder(categories=kategori_urutan)
data['Transportation'] = encoder.fit_transform(data[['Transportation']])

data

,Unnamed: 0,Id,Student_Age,Sex,High_School_Type,Additional_Work,Sports_activity,Transportation,Weekly_Study_Hours,Attendance,Reading,Notes,Listening_in_Class,Project_work,Grade,Scholarship
0,0,5001,21,0.0,2.0,1,0,0.0,0,Always,1,2,1,0,AA,2
1,1,5002,20,0.0,2.0,1,0,0.0,0,Always,1,1,2,1,AA,2
2,2,5003,21,0.0,0.0,0,0,0.0,2,Never,0,1,1,1,AA,2
3,3,5004,18,1.0,1.0,1,0,1.0,2,Always,0,2,1,0,AA,2
4,4,5005,22,0.0,1.0,0,0,1.0,12,Always,1,1,2,1,AA,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,140,5141,22,1.0,0.0,1,1,0.0,0,Always,0,2,1,1,CC,2
141,141,5142,18,1.0,0.0,0,0,0.0,0,Never,0,2,2,0,CC,3
142,142,5143,18,1.0,1.0,0,0,0.0,0,Always,1,1,1,0,AA,3
143,143,5144,22,1.0,0.0,1,1,1.0,12,Sometimes,0,2,1,1,CB,3


In [ ]:
kategori_urutan = [['Always', 'Never', 'Sometimes', '3']]
encoder = OrdinalEncoder(categories=kategori_urutan)
data['Attendance'] = encoder.fit_transform(data[['Attendance']])

data

,Unnamed: 0,Id,Student_Age,Sex,High_School_Type,Additional_Work,Sports_activity,Transportation,Weekly_Study_Hours,Attendance,Reading,Notes,Listening_in_Class,Project_work,Grade,Scholarship
0,0,5001,21,0.0,2.0,1,0,0.0,0,0.0,1,2,1,0,AA,2
1,1,5002,20,0.0,2.0,1,0,0.0,0,0.0,1,1,2,1,AA,2
2,2,5003,21,0.0,0.0,0,0,0.0,2,1.0,0,1,1,1,AA,2
3,3,5004,18,1.0,1.0,1,0,1.0,2,0.0,0,2,1,0,AA,2
4,4,5005,22,0.0,1.0,0,0,1.0,12,0.0,1,1,2,1,AA,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,140,5141,22,1.0,0.0,1,1,0.0,0,0.0,0,2,1,1,CC,2
141,141,5142,18,1.0,0.0,0,0,0.0,0,1.0,0,2,2,0,CC,3
142,142,5143,18,1.0,1.0,0,0,0.0,0,0.0,1,1,1,0,AA,3
143,143,5144,22,1.0,0.0,1,1,1.0,12,2.0,0,2,1,1,CB,3


In [ ]:
data['Grade'] = data['Grade'].apply(lambda x: 1 if x != 'Fail' else 0)
data

,Unnamed: 0,Id,Student_Age,Sex,High_School_Type,Additional_Work,Sports_activity,Transportation,Weekly_Study_Hours,Attendance,Reading,Notes,Listening_in_Class,Project_work,Grade,Scholarship
0,0,5001,21,0.0,2.0,1,0,0.0,0,0.0,1,2,1,0,1,2
1,1,5002,20,0.0,2.0,1,0,0.0,0,0.0,1,1,2,1,1,2
2,2,5003,21,0.0,0.0,0,0,0.0,2,1.0,0,1,1,1,1,2
3,3,5004,18,1.0,1.0,1,0,1.0,2,0.0,0,2,1,0,1,2
4,4,5005,22,0.0,1.0,0,0,1.0,12,0.0,1,1,2,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,140,5141,22,1.0,0.0,1,1,0.0,0,0.0,0,2,1,1,1,2
141,141,5142,18,1.0,0.0,0,0,0.0,0,1.0,0,2,2,0,1,3
142,142,5143,18,1.0,1.0,0,0,0.0,0,0.0,1,1,1,0,1,3
143,143,5144,22,1.0,0.0,1,1,1.0,12,2.0,0,2,1,1,1,3


# Handling Outlier

In [ ]:
# pip install matplotlib
import matplotlib
print(matplotlib.__version__)

import matplotlib.pyplot as plt
plt.switch_backend('agg')


3.7.1


In [ ]:
plt.figure(figsize=(20, 10))
data.boxplot(vert=False)

<Axes: >

In [ ]:
winsorizer = Winsorizer(
    capping_method='iqr',
    tail='both',
    fold=1.5,
    variables=['Weekly_Study_Hours', 'Student_Age', 'Attendance', 'Scholarship'])

winsorizer.fit(data)

data = winsorizer.transform(data)


In [ ]:
plt.figure(figsize=(20, 10))
data.boxplot(vert=False)

<Axes: >

In [ ]:
cleaned_data = data.drop(columns=['Id', 'Unnamed: 0'])
corr = cleaned_data.corr()
corr

,Student_Age,Sex,High_School_Type,Additional_Work,Sports_activity,Transportation,Weekly_Study_Hours,Attendance,Reading,Notes,Listening_in_Class,Project_work,Grade,Scholarship
Student_Age,1.000000,-0.112538,-0.052822,0.158747,-0.116122,-0.065638,0.167289,-0.092162,0.009602,0.185459,-0.106092,0.044514,0.113922,-0.064709
Sex,-0.112538,1.000000,0.048980,0.190476,0.224138,-0.125471,-0.004495,0.234087,-0.045101,0.053468,0.010966,0.118254,-0.295958,-0.039667
High_School_Type,-0.052822,0.048980,1.000000,0.022449,-0.053062,-0.117883,0.181777,0.026563,-0.001519,0.009166,-0.118969,0.094052,0.054964,-0.071422
Additional_Work,0.158747,0.190476,0.022449,1.000000,0.220238,0.100010,0.081549,0.086283,-0.096846,0.211961,0.113379,-0.126299,-0.146658,0.017493
Sports_activity,-0.116122,0.224138,-0.053062,0.220238,1.000000,-0.011406,-0.116866,0.153675,-0.186040,0.133669,-0.071278,0.090098,-0.110984,-0.091406
Transportation,-0.065638,-0.125471,-0.117883,0.100010,-0.011406,1.000000,0.064652,0.161001,-0.112662,0.011893,0.105256,-0.036034,-0.161196,0.001829
Weekly_Study_Hours,0.167289,-0.004495,0.181777,0.081549,-0.116866,0.064652,1.000000,0.103725,0.072169,-0.038933,-0.038249,-0.013820,-0.114516,0.087427
Attendance,-0.092162,0.234087,0.026563,0.086283,0.153675,0.161001,0.103725,1.000000,-0.420264,-0.108345,0.016871,0.367741,-0.304620,0.066696
Reading,0.009602,-0.045101,-0.001519,-0.096846,-0.186040,-0.112662,0.072169,-0.420264,1.000000,-0.044310,0.021143,-0.255810,0.109286,-0.129274
Notes,0.185459,0.053468,0.009166,0.211961,0.133669,0.011893,-0.038933,-0.108345,-0.044310,1.000000,-0.007036,-0.137294,0.065275,-0.037346


In [ ]:
plt.figure(figsize=(16, 6))
sns.heatmap(corr, annot=True, fmt=".2f")
plt.show()

Random Forest

In [ ]:
# Pisahkan fitur prediktor (X) dan target (y)
X = cleaned_data.drop('Grade', axis=1)
y = cleaned_data['Grade']

X_encoded = pd.get_dummies(X)

# Bagi data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inisialisasi model Random Forest
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Latih model dengan data latih
rf_classifier.fit(X_train, y_train)

# Prediksi pada data uji
y_pred = rf_classifier.predict(X_test)

# Hitung akurasi
accuracy = accuracy_score(y_test, y_pred)
print(f"Akurasi: {accuracy}")

# Tampilkan classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

Akurasi: 0.8620689655172413
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         4
           1       0.86      1.00      0.93        25

    accuracy                           0.86        29
   macro avg       0.43      0.50      0.46        29
weighted avg       0.74      0.86      0.80        29



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Prediksi pada data uji
y_pred = rf_classifier.predict(X_test)

# Hitung confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Mendapatkan label kelas yang unik dari data uji
classes = rf_classifier.classes_

# Menampilkan confusion matrix secara visual
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=classes)
disp.plot(cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()


KNN (K-nearest neighbors)

In [ ]:
# Pisahkan fitur prediktor (X) dan target (y)
X = cleaned_data.drop('Grade', axis=1)
y = cleaned_data['Grade']

# Bagi data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inisialisasi model KNN
knn_classifier = KNeighborsClassifier()

# Lakukan cross-validation untuk mencari nilai k terbaik
k_values = list(range(1, 31))
cv_scores = []

for k in k_values:
    knn_classifier = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn_classifier, X_train, y_train, cv=5, scoring='accuracy')
    cv_scores.append(scores.mean())

# Temukan nilai k terbaik
optimal_k = k_values[cv_scores.index(max(cv_scores))]
print(f"Nilai k terbaik: {optimal_k}")

# Inisialisasi model KNN dengan nilai k terbaik
knn_classifier = KNeighborsClassifier(n_neighbors=optimal_k)

# Latih model dengan data latih
knn_classifier.fit(X_train, y_train)

# Prediksi pada data uji
y_pred = knn_classifier.predict(X_test)

# Hitung akurasi
accuracy = accuracy_score(y_test, y_pred)
print(f"Akurasi: {accuracy}")

# Tampilkan classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklea

Nilai k terbaik: 3
Akurasi: 0.8620689655172413
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         4
           1       0.86      1.00      0.93        25

    accuracy                           0.86        29
   macro avg       0.43      0.50      0.46        29
weighted avg       0.74      0.86      0.80        29



/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predic

Hyperparameter Tuning with GridSearchCV

In [ ]:
# Pisahkan fitur prediktor (X) dan target (y)
X = cleaned_data.drop('Grade', axis=1)
y = cleaned_data['Grade']

# Bagi data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inisialisasi model Random Forest
rf_classifier = RandomForestClassifier(random_state=42)

# Definisikan parameter grid untuk tuning
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Lakukan GridSearchCV
grid_search = GridSearchCV(estimator=rf_classifier,
                           param_grid=param_grid,
                           cv=5,
                           scoring='accuracy',
                           n_jobs=-1,
                           verbose=1)

# Fit grid search pada data latih
grid_search.fit(X_train, y_train)

# Tampilkan parameter terbaik
print("Parameter terbaik:", grid_search.best_params_)

# Buat model dengan parameter terbaik
rf_best = grid_search.best_estimator_

# Evaluasi model dengan data uji
y_pred = rf_best.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Akurasi: {accuracy}")

print("Classification Report:")
print(classification_report(y_test, y_pred))

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


Fitting 5 folds for each of 81 candidates, totalling 405 fits
Parameter terbaik: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 100}
Akurasi: 0.8620689655172413
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         4
           1       0.86      1.00      0.93        25

    accuracy                           0.86        29
   macro avg       0.43      0.50      0.46        29
weighted avg       0.74      0.86      0.80        29



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# Kesimpulan

1. Menggunakan dua metode klasifikasi, yaitu Random Forest dan K-Nearest Neighbors (KNN), telah digunakan untuk membangun model prediksi.

2. Akurasi tertinggi yang diperoleh dari kedua metode tersebut adalah 0,8620689655172413 atau sekitar 86,21%.


3. Untuk kelas mayoritas (kelas 1) dengan 25 sampel, nilai recall adalah 1. Ini berarti semua sampel dari kelas 1 diprediksi dengan benar oleh model.

4. Hyperparameter tuning dengan GridSearchCV telah dilakukan untuk Random Forest. Parameter terbaik yang ditemukan adalah 'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 5, dan 'n_estimators': 100.
